In [1]:
import os
os.chdir("../src/")
import pandas as pd
import s3fs
import import_inpi as inpi

In [3]:
pm, rep = inpi.import_inpi_s3()

/home/coder/work/data_inpi/src/import_inpi.py:109: DtypeWarning: Columns (7,8,10,11,16,30) have mixed types. Specify dtype option on import or set low_memory=False.
  personnes_morales = pd.read_csv(file_in, sep=";")
/home/coder/work/data_inpi/src/import_inpi.py:111: DtypeWarning: Columns (7,15,17,25,26,32,33,34,35,36,37,38,39,40,41,42,43,45,49,50) have mixed types. Specify dtype option on import or set low_memory=False.
  representants = pd.read_csv(file_in, sep=";")


In [4]:
pm.shape

(5001845, 31)

In [5]:
rep.shape

(11142886, 51)